In [ ]:
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd

In [ ]:
user_data_dir = Path("/Volumes/One Touch/Downloads/files")
all_user_data = list(user_data_dir.glob("0*.parquet"))
ny_position_data_file = Path("ny_positions.csv")
result_dir = Path("filtered_user_data")
result_dir.mkdir(exist_ok=True)

In [ ]:
ny_position_data = pd.read_csv(ny_position_data_file)
ny_position_data["user_id"] = ny_position_data["user_id"].astype(int).astype(str)
ny_position_data.drop_duplicates(subset=["user_id"], inplace=True)

In [ ]:
# show how many row groups are in the parquet file
for parquet_file in all_user_data:
    output_file = result_dir / parquet_file.name.replace(".parquet", ".csv")
    if output_file.exists():
        continue
    print("processing ", parquet_file)
    data = pq.ParquetFile(parquet_file)
    n_row_groups = data.num_row_groups
    all_merged_data = []
    for ri in range(n_row_groups):
        if ri % 20 == 0:
            print("processing row group ", ri, " out of ", n_row_groups)
        table = data.read_row_group(ri).to_pandas()
        table["user_id"] = table["user_id"].astype(str)
        new_marged_table = pd.merge(table, ny_position_data, on="user_id", how="inner")
        all_merged_data.append(new_marged_table)
    print(len(all_merged_data))
    all_merged_data = pd.concat(all_merged_data)
    all_merged_data.to_csv(result_dir / parquet_file.name.replace(".parquet", ".csv"), index=False)
        
        

In [2]:
import pandas as pd
from pathlib import  Path

In [ ]:
user_data_dir = Path("filtered_user_data")
user_data_files = list(user_data_dir.glob("*.csv"))
# merge all data into one dataframe, only columns [`user_id`,`firstname`,`lastname`]
all_data = []
for user_data_f in user_data_files:
    # print(f"Reading {user_data_f}")
    user_data = pd.read_csv(user_data_f, usecols=["user_id", "firstname", "lastname", "IBES_id", "final_parent_factset_name_y"])
    all_data.append(user_data)
all_data = pd.concat(all_data, ignore_index=True)
print(all_data.shape)
# replace `-` with space
# add a new column `full_name` which is the concatenation of `firstname` and `lastname`
# clean first name and last name, lower case, remove leading and trailing spaces
all_data["firstname"] = all_data["firstname"].str.replace("-", " ")
all_data["lastname"] = all_data["lastname"].str.replace("-", " ")
all_data["firstname_cleaned"] = all_data["firstname"].str.strip().str.lower()
all_data["lastname_cleaned"] = all_data["lastname"].str.strip().str.lower()

all_data["full_name"] = all_data["firstname_cleaned"] + " " + all_data["lastname_cleaned"]
# remove duplicates, only keep unique `full_name`
# if a user has multiple `full_name`, remove them all
all_data_noduplicate = all_data.drop_duplicates(subset=["full_name"], keep=False)
full_name = all_data_noduplicate[["full_name", "user_id"]]
full_name.to_csv("filtered_user_unique_full_name.csv", index=False)
full_name.shape

(362656, 3)


KeyError: "['IBES_id', 'final_parent_factset_name_y'] not in index"